# Predicting reaction performance in C–N cross-coupling using machine learning

{{ badge }}

DOI: 10.1126/science.aar5169

Ahneman, D. T.; Estrada, J. G.; Lin, S.; Dreher, S. D.; Doyle, A. G. *Science*, **2018**, *360*, 186-190.

**Comments/Questions:**
- For this type of screening it is pretty reasonable to simply report stock solution contents and molarity. So not surprising they didn't specify. Should there be a concentration option? Here I wrote a class to handle stock solutions and infer moles and volume from the reported concentration. 
- They worked with stock solutions (and most screens do) - does the API keep track of total volume?
- How do you want to handle internal/external standards? - Here I add another component to the workup.
- Analysis details should definitely have HPLC and UPLC as an option. Here I used LCMS instead.
- I was having a little trouble getting replacement to work from the template so I just looped through the whole setup.
- This took quite a while to run on my laptop (i7-6500U): 6089 s. The majority of this came from validation: ~ 2 s per reaction.


Import schema and helper functions

In [1]:
import ord_schema
from datetime import datetime
from ord_schema.proto import reaction_pb2
from ord_schema.units import UnitResolver
from ord_schema import validations
from ord_schema import message_helpers

unit_resolver = UnitResolver()

ImportError: cannot import name '_message' from 'google.protobuf.pyext' (C:\Users\Ben\Anaconda3\envs\ord\lib\site-packages\google\protobuf\pyext\__init__.py)

# Define a single reaction

Single reaction from the SI to be used as a template for the remaining entries.

Start by writing a helper function for defining stock solutions.

In [3]:
class stock_solution:
    """Helper class for defining stock solutions."""
    
    def __init__(self, reaction, stock_name):
        self.stock = reaction.inputs[stock_name]
        self.concentration = 0
        self.moles = 0
        self.volume = 0

    def add_solute(self, role, name, SMILES=None, is_limiting=False, preparation='NONE',
                   moles=0, volume_liters=0):
        """Add solute to solution. Keep track of moles of solute and total volume."""
        
        # Solution volume is sum of solute and solvent volumes
        self.moles += moles
        self.volume += volume_liters
        
        # Add solute and ID
        self.solute = self.stock.components.add()
        self.solute.reaction_role = reaction_pb2.Compound.ReactionRole.__dict__[role]
        self.solute.identifiers.add(value=name, type='NAME')
        if SMILES != None:
            self.solute.identifiers.add(value=SMILES, type='SMILES')
               
        # Other details
        self.solute.preparation.type = reaction_pb2.CompoundPreparation.PreparationType.Value(preparation)
        self.solute.is_limiting = is_limiting
        
    def add_solvent(self, name,  SMILES=None, preparation='NONE', volume_liters=0):
        """Add solvent to solution. Keep track of total volume."""
        
        # Solution volume is sum of solute and solvent volumes
        self.volume += volume_liters
        
        # Add solute and ID
        self.solvent = self.stock.components.add()
        self.solvent.reaction_role = reaction_pb2.Compound.ReactionRole.SOLVENT
        self.solvent.identifiers.add(value=name, type='NAME')
        if SMILES != None:
            self.solute.identifiers.add(value=SMILES, type='SMILES')
        
        # Other details
        self.solvent.preparation.type = reaction_pb2.CompoundPreparation.PreparationType.Value(preparation)
        
    def mix(self, concentration_molar=0):
        """Mix function resolves moles and volume from availible information (concentration, moles, volume)"""
        
        self.concentration = concentration_molar
        
        # Resolve concentration
        if self.moles > 0 and self.volume > 0:
            self.solute.moles.CopyFrom(unit_resolver.resolve(str(int(self.moles)) + ' mol'))
            self.solvent.volume.CopyFrom(unit_resolver.resolve(str(int(self.volume)) + ' L'))
        elif self.concentration > 0 and self.volume > 0:
            self.moles = self.concentration * self.volume
            self.solute.moles.CopyFrom(unit_resolver.resolve(str(int(self.moles)) + ' mol'))
            self.solvent.volume.CopyFrom(unit_resolver.resolve(str(int(self.volume)) + ' L'))


**Define reaction inputs**:
- Catalyst in DMSO (0.05 M)
- Electrophile in DMSO (0.50 M)
- Nucleophile in DMSO (0.50 M)
- Additive in DMSO (0.50 M)
- Base in DMSO (0.75 M)
- The SI does not indicate an order of addition


In [4]:
# Define Reaction
reaction = reaction_pb2.Reaction()
reaction.identifiers.add(value=r'Buchwald-Hartwig Amination', type='NAME')

# Catalyst stock solution
catalyst = stock_solution(reaction, r'Pd precatalyst in DMSO')
catalyst.add_solute('CATALYST', r'XPhos', SMILES=r'CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)C4CCCCC4)C=CC=C2')
catalyst.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
catalyst.mix(concentration_molar=0.05)

# Electrophile stock solution
electrophile = stock_solution(reaction, r'Aryl halide in DMSO')
electrophile.add_solute('REACTANT', r'4-trifuloromethyl chlorobenzene', SMILES=r'ClC1=CC=C(C(F)(F)F)C=C1', is_limiting=True)
electrophile.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
electrophile.mix(concentration_molar=0.50)

# Nucleophile stock solution
nucleophile = stock_solution(reaction, r'Amine in DMSO')
nucleophile.add_solute('REACTANT', r'p-toluidine', SMILES=r'NC1=CC=C(C)C=C1')
nucleophile.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
nucleophile.mix(concentration_molar=0.50)

# Additive stock solution
additive = stock_solution(reaction, r'Additive in DMSO')
additive.add_solute('REAGENT', r'5-phenylisoxazole', SMILES=r'o1nccc1c2ccccc2')
additive.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
additive.mix(concentration_molar=0.50)

# Base stock solution 
base = stock_solution(reaction, r'Base in DMSO')
base.add_solute('REAGENT', r'P2Et', SMILES=r'CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC')
base.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
base.mix(concentration_molar=0.75)

Define reaction setup & conditions

In [5]:
# Reactions performed in 1556 well plate
reaction.setup.vessel.CopyFrom(
    reaction_pb2.Vessel(
        type='WELL_PLATE', 
        material='PLASTIC',
        volume=unit_resolver.resolve('12.5 uL')
    )
)
reaction.setup.is_automated = True

In [6]:
# Heated - not specified how
t_conds = reaction.conditions.temperature
t_conds.setpoint.CopyFrom(reaction_pb2.Temperature(units='CELSIUS', value=60))

In [7]:
# Glove box work
p_conds = reaction.conditions.pressure
p_conds.type = p_conds.PressureControl.SEALED
p_conds.atmosphere = p_conds.Atmosphere.NITROGEN 
p_conds.atmosphere_details = 'dry nitrogen'
p_conds.details = 'prepared in a glove box'

In [9]:
# No safety notes
reaction.notes.safety_notes = ''

After 16 h, the plate was opened and the Mosquito was used to add internal standard to each well (3 µL of 0.0025 M di-tert-butylbiphenyl solution in DMSO). At that point, aliquots were sampled into 384-well plates and analyzed by UPLC.

In [10]:
# Standard stock solution 
standard = stock_solution(reaction, r'External standard in DMSO')
standard.add_solute('WORKUP', r'4,4\'-di-tert-butyl-1,1\'-biphenyl', SMILES=r'CC(C)(C)C1=CC=C(C2=CC=C(C(C)(C)C)C=C2)C=C1')
standard.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=3e-6)
standard.mix(concentration_molar=0.0025)

In [11]:
outcome = reaction.outcomes.add()
outcome.reaction_time.CopyFrom(unit_resolver.resolve('16 hrs'))

# Analyses: UPLC
# Note using LCMS because UPLC is not an option
outcome.analyses['UPLC analysis'].type = reaction_pb2.ReactionAnalysis.LCMS
outcome.analyses['UPLC analysis'].details = ('UPLC using 3 µL of 0.0025 M di-tert-butylbiphenyl solution in DMSO external standard')
outcome.analyses['UPLC analysis'].instrument_manufacturer = 'Waters Acquity'

# Define product identity
prod_2a = outcome.products.add() 
prod_2a.compound.identifiers.add().CopyFrom(
    reaction_pb2.CompoundIdentifier(value=r'FC(C1=CC=C(NC2=CC=C(C)C=C2)C=C1)(F)F', type='SMILES')
)
prod_2a.is_desired_product = True

# Define product yield from results table
# Yields are reported with insignificant digits
prod_2a.compound_yield.CopyFrom(
    reaction_pb2.Percentage(value=10.65781182) 
)

# The UPLC analysis was used to confirm both identity and yield
prod_2a.analysis_identity.append('UPLC analysis')
prod_2a.analysis_yield.append('UPLC analysis')

# Reaction provenance
reaction.provenance.city = r'Kenilworth, NJ'
reaction.provenance.doi = r'10.1126/science.aar5169'
reaction.provenance.publication_url = r'https://science.sciencemag.org/content/360/6385/186'
reaction.provenance.record_created.time.value = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
reaction.provenance.record_created.person.CopyFrom(reaction_pb2.Person(
    name='Benjamin J. Shields', organization='Princeton University'))

Validate and examine this final prototypical reaction entry

In [12]:
validations.validate_message(reaction)

identifiers {
  type: NAME
  value: "Buchwald-Hartwig Amination"
}
inputs {
  key: "Additive in DMSO"
  value {
    components {
      identifiers {
        type: NAME
        value: "5-phenylisoxazole"
      }
      identifiers {
        type: SMILES
        value: "o1nccc1c2ccccc2"
      }
      identifiers {
        type: SMILES
        value: "O=S(C)C"
      }
      identifiers {
        type: RDKIT_BINARY
        bytes_value: "\357\276\255\336\000\000\000\000\013\000\000\000\000\000\000\000\000\000\000\000\013\000\000\000\014\000\000\000\200\001\010@(\000\000\000\003\002\007@(\000\000\000\003\003\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\006@(\000\000\000\003\004\006@(\000\000\000\003\004\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\013\000\001h\014\001\002h\014\002\003h\014\003\004h\014\004\005 \005\006h\014\006\007h\014\007\010h\014\010\th\014\t\nh\014\004\0

# Full HTE Data Set

In [13]:
# Get full set of reactions: I preprocessed this to have SMILES for each component.
# Note I am only including the data that was used for modeling - there are some
#     controls and failed reactions in the SI (if we even want them?).

import pandas as pd

index = pd.read_csv('experiment_index.csv')
index

,entry,Aryl_halide_SMILES,Additive_SMILES,Base_SMILES,Ligand_SMILES,yield
0,49,FC(F)(F)c1ccc(Cl)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,10.657812
1,50,FC(F)(F)c1ccc(Br)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,14.747896
2,51,FC(F)(F)c1ccc(I)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,18.278686
3,52,COc1ccc(Cl)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,2.475058
4,53,COc1ccc(Br)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,6.119058
...,...,...,...,...,...,...
3950,4603,Brc1ccccn1,COC(=O)c1cc(on1)c2sccc2,CN1CCCN2CCCN=C12,CC(C1=C(C2=C(OC)C=CC(OC)=C2P(C34CC5CC(C4)CC(C5...,57.426670
3951,4604,Ic1ccccn1,COC(=O)c1cc(on1)c2sccc2,CN1CCCN2CCCN=C12,CC(C1=C(C2=C(OC)C=CC(OC)=C2P(C34CC5CC(C4)CC(C5...,86.233157
3952,4605,Clc1cccnc1,COC(=O)c1cc(on1)c2sccc2,CN1CCCN2CCCN=C12,CC(C1=C(C2=C(OC)C=CC(OC)=C2P(C34CC5CC(C4)CC(C5...,1.440081
3953,4606,Brc1cccnc1,COC(=O)c1cc(on1)c2sccc2,CN1CCCN2CCCN=C12,CC(C1=C(C2=C(OC)C=CC(OC)=C2P(C34CC5CC(C4)CC(C5...,43.538365


In [14]:
# I happened to have ID tables around so we can give the components names

def match_name(column, list_path):
    """Match names from csv files to SMILES."""
    
    component_list = pd.read_csv(list_path)
    
    # Get SMILES column
    for col in component_list.columns.values:
        if 'SMILES' in col:
            smi_col = col
            
    # Get name column
    names = index[column].copy()
    for i in range(len(component_list)):
        names = names.replace(component_list[smi_col][i], component_list['name'][i])
    
    return names.values

index['Aryl_halide_name'] = match_name('Aryl_halide_SMILES', 'aryl_halide-list.csv')
index['Additive_name'] = match_name('Additive_SMILES', 'additive-list.csv')
index['Base_name'] = match_name('Base_SMILES', 'base-list.csv')
index['Ligand_name'] = match_name('Ligand_SMILES', 'ligand-list.csv')

index.head()

,entry,Aryl_halide_SMILES,Additive_SMILES,Base_SMILES,Ligand_SMILES,yield,Aryl_halide_name,Additive_name,Base_name,Ligand_name
0,49,FC(F)(F)c1ccc(Cl)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,10.657812,1-chloro-4-(trifluoromethyl)benzene,5-phenylisoxazole,P2Et,X-Phos
1,50,FC(F)(F)c1ccc(Br)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,14.747896,1-bromo-4-(trifluoromethyl)benzene,5-phenylisoxazole,P2Et,X-Phos
2,51,FC(F)(F)c1ccc(I)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,18.278686,1-iodo-4-(trifluoromethyl)benzene,5-phenylisoxazole,P2Et,X-Phos
3,52,COc1ccc(Cl)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,2.475058,1-chloro-4-methoxybenzene,5-phenylisoxazole,P2Et,X-Phos
4,53,COc1ccc(Br)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,6.119058,1-bromo-4-methoxybenzene,5-phenylisoxazole,P2Et,X-Phos


In [15]:
# Products aren't listed - Use rdkit to get them

from rdkit import Chem
from rdkit.Chem import AllChem

def amination(aryl_halide):
    """Get product based on aryl halide identity."""
    
    replace_with = Chem.MolFromSmiles('NC1=CC=C(C)C=C1')
    pattern = Chem.MolFromSmarts('[Cl,Br,I]')
    molecule = Chem.MolFromSmiles(aryl_halide)
    product = AllChem.ReplaceSubstructs(molecule, pattern, replace_with)
    
    return Chem.MolToSmiles(product[0])
    
index['Product_SMILES'] = [amination(aryl_halide) for aryl_halide in index['Aryl_halide_SMILES'].tolist()]

index.head()

,entry,Aryl_halide_SMILES,Additive_SMILES,Base_SMILES,Ligand_SMILES,yield,Aryl_halide_name,Additive_name,Base_name,Ligand_name,Product_SMILES
0,49,FC(F)(F)c1ccc(Cl)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,10.657812,1-chloro-4-(trifluoromethyl)benzene,5-phenylisoxazole,P2Et,X-Phos,Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1
1,50,FC(F)(F)c1ccc(Br)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,14.747896,1-bromo-4-(trifluoromethyl)benzene,5-phenylisoxazole,P2Et,X-Phos,Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1
2,51,FC(F)(F)c1ccc(I)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,18.278686,1-iodo-4-(trifluoromethyl)benzene,5-phenylisoxazole,P2Et,X-Phos,Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1
3,52,COc1ccc(Cl)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,2.475058,1-chloro-4-methoxybenzene,5-phenylisoxazole,P2Et,X-Phos,COc1ccc(Nc2ccc(C)cc2)cc1
4,53,COc1ccc(Br)cc1,o1nccc1c2ccccc2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,6.119058,1-bromo-4-methoxybenzene,5-phenylisoxazole,P2Et,X-Phos,COc1ccc(Nc2ccc(C)cc2)cc1


In [16]:
# Reorder the dataframe
index = index[['Ligand_SMILES', 'Ligand_name',
               'Aryl_halide_SMILES', 'Aryl_halide_name',
               'Additive_SMILES', 'Additive_name',
               'Base_SMILES', 'Base_name',
               'Product_SMILES', 'yield']]

In [17]:
# Gonna time execution

import time

class timer:
    """
    Returns wall clock-time
    """
    
    def __init__(self, name):
        
        self.start = time.time()
        self.name = name
        
    def stop(self):
        self.end = time.time()    
        print(self.name + ': ' + str(self.end - self.start) + ' s')

The only aspects of reaction data that vary are: (1) ligand, (2) electrophile, (3) additive, and (4) base.

In [31]:
t = timer('30 Entries')

# Testing takes > 6000 s with full reaction set - here I will use random sample of 30 experiments
# To run all experiments remove sample(30) below

reactions = []
for lig_s, lig_n, elec_s, elec_n, add_s, add_n, base_s, base_n, prod, y in index.sample(30).values:  # Random Sample
    
    # Define Reaction
    reaction = reaction_pb2.Reaction()
    reaction.identifiers.add(value=r'Buchwald-Hartwig Amination', type='NAME')

    # Catalyst stock solution
    catalyst = stock_solution(reaction, r'Pd precatalyst in DMSO')
    catalyst.add_solute('CATALYST', lig_n, SMILES=lig_s)
    catalyst.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    catalyst.mix(concentration_molar=0.05)

    # Electrophile stock solution
    electrophile = stock_solution(reaction, r'Aryl halide in DMSO')
    electrophile.add_solute('REACTANT', elec_n, SMILES=elec_s, is_limiting=True)
    electrophile.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    electrophile.mix(concentration_molar=0.50)

    # Nucleophile stock solution
    nucleophile = stock_solution(reaction, r'Amine in DMSO')
    nucleophile.add_solute('REACTANT', r'p-toluidine', SMILES=r'NC1=CC=C(C)C=C1')
    nucleophile.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    nucleophile.mix(concentration_molar=0.50)

    # Additive stock solution
    additive = stock_solution(reaction, r'Additive in DMSO')
    additive.add_solute('REAGENT', add_n, SMILES=add_s)
    additive.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    additive.mix(concentration_molar=0.50)

    # Base stock solution 
    base = stock_solution(reaction, r'Base in DMSO')
    base.add_solute('REAGENT', base_n, SMILES=base_s)
    base.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    base.mix(concentration_molar=0.75)

    # Reactions performed in 1556 well plate
    reaction.setup.vessel.CopyFrom(
        reaction_pb2.Vessel(
            type='WELL_PLATE', 
            material='PLASTIC',
            volume=unit_resolver.resolve('12.5 uL')
        )
    )
    reaction.setup.is_automated = True

    # Heated - not specified how
    t_conds = reaction.conditions.temperature
    t_conds.setpoint.CopyFrom(reaction_pb2.Temperature(units='CELSIUS', value=60))

    # Glove box work
    p_conds = reaction.conditions.pressure
    p_conds.type = p_conds.PressureControl.SEALED
    p_conds.atmosphere = p_conds.Atmosphere.NITROGEN 
    p_conds.atmosphere_details = 'dry nitrogen'
    p_conds.details = 'prepared in a glove box'

    # Notes
    reaction.notes.safety_notes = ''

    # Standard stock solution 
    standard = stock_solution(reaction, r'External standard in DMSO')
    standard.add_solute('WORKUP', r'4,4\'-di-tert-butyl-1,1\'-biphenyl', SMILES=r'CC(C)(C)C1=CC=C(C2=CC=C(C(C)(C)C)C=C2)C=C1')
    standard.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=3e-6)
    standard.mix(concentration_molar=0.0025)

    outcome = reaction.outcomes.add()
    outcome.reaction_time.CopyFrom(unit_resolver.resolve('16 hrs'))

    # Analyses: UPLC
    # Note using LCMS because UPLC is not an option
    outcome.analyses['UPLC analysis'].type = reaction_pb2.ReactionAnalysis.LCMS
    outcome.analyses['UPLC analysis'].details = ('UPLC using 3 µL of 0.0025 M di-tert-butylbiphenyl solution in DMSO external standard')
    outcome.analyses['UPLC analysis'].instrument_manufacturer = 'Waters Acquity'

    # Define product identity
    prod_2a = outcome.products.add() 
    prod_2a.compound.identifiers.add().CopyFrom(
        reaction_pb2.CompoundIdentifier(value=prod, type='SMILES')
    )
    prod_2a.is_desired_product = True

    # Define product yield from results table
    # Yields are reported with insignificant digits
    prod_2a.compound_yield.CopyFrom(
        reaction_pb2.Percentage(value=round(y))    # Round yield to the nearest % 
    )

    # The UPLC analysis was used to confirm both identity and yield
    prod_2a.analysis_identity.append('UPLC analysis')
    prod_2a.analysis_yield.append('UPLC analysis')

    # Reaction provenance
    reaction.provenance.city = r'Kenilworth, NJ'
    reaction.provenance.doi = r'10.1126/science.aar5169'
    reaction.provenance.publication_url = r'https://science.sciencemag.org/content/360/6385/186'
    reaction.provenance.record_created.time.value = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    reaction.provenance.record_created.person.CopyFrom(reaction_pb2.Person(
        name='Benjamin J. Shields', organization='Princeton University')
    )
    
    # Validate
    reaction = validations.validate_message(reaction)
    
    # Append
    reactions.append(reaction)

t.stop()

3955 Entries: 6088.877344608307 s


In [32]:
print(f'Generated {len(reactions)} reactions')

Generated 3955 reactions


In [33]:
# Inspect random reaction from this set
reactions[42]

identifiers {
  type: NAME
  value: "Buchwald-Hartwig Amination"
}
inputs {
  key: "Additive in DMSO"
  value {
    components {
      identifiers {
        type: NAME
        value: "5-phenylisoxazole"
      }
      identifiers {
        type: SMILES
        value: "o1nccc1c2ccccc2"
      }
      identifiers {
        type: SMILES
        value: "O=S(C)C"
      }
      identifiers {
        type: RDKIT_BINARY
        bytes_value: "\357\276\255\336\000\000\000\000\013\000\000\000\000\000\000\000\000\000\000\000\013\000\000\000\014\000\000\000\200\001\010@(\000\000\000\003\002\007@(\000\000\000\003\003\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\006@(\000\000\000\003\004\006@(\000\000\000\003\004\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\006@h\000\000\000\003\003\001\013\000\001h\014\001\002h\014\002\003h\014\003\004h\014\004\005 \005\006h\014\006\007h\014\007\010h\014\010\th\014\t\nh\014\004\0

That took a really long time. Let's see why...

In [29]:
for lig_s, lig_n, elec_s, elec_n, add_s, add_n, base_s, base_n, prod, y in index.values[:2]:
    ###
    t = timer('Instantiate reaction')
    # Define Reaction
    reaction = reaction_pb2.Reaction()
    reaction.identifiers.add(value=r'Buchwald-Hartwig Amination', type='NAME')
    t.stop()
    
    ###
    t = timer('Stock solutions')
    # Catalyst stock solution
    catalyst = stock_solution(reaction, r'Pd precatalyst in DMSO')
    catalyst.add_solute('CATALYST', lig_n, SMILES=lig_s)
    catalyst.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    catalyst.mix(concentration_molar=0.05)

    # Electrophile stock solution
    electrophile = stock_solution(reaction, r'Aryl halide in DMSO')
    electrophile.add_solute('REACTANT', elec_n, SMILES=elec_s, is_limiting=True)
    electrophile.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    electrophile.mix(concentration_molar=0.50)

    # Nucleophile stock solution
    nucleophile = stock_solution(reaction, r'Amine in DMSO')
    nucleophile.add_solute('REACTANT', r'p-toluidine', SMILES=r'NC1=CC=C(C)C=C1')
    nucleophile.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    nucleophile.mix(concentration_molar=0.50)

    # Additive stock solution
    additive = stock_solution(reaction, r'Additive in DMSO')
    additive.add_solute('REAGENT', add_n, SMILES=add_s)
    additive.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    additive.mix(concentration_molar=0.50)

    # Base stock solution 
    base = stock_solution(reaction, r'Base in DMSO')
    base.add_solute('REAGENT', base_n, SMILES=base_s)
    base.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=200e-9)
    base.mix(concentration_molar=0.75)
    t.stop()
    
    ###
    t = timer('Reaction setup')
    # Reactions performed in 1556 well plate
    reaction.setup.vessel.CopyFrom(
        reaction_pb2.Vessel(
            type='WELL_PLATE', 
            material='PLASTIC',
            volume=unit_resolver.resolve('12.5 uL')
        )
    )
    reaction.setup.is_automated = True

    # Heated - not specified how
    t_conds = reaction.conditions.temperature
    t_conds.setpoint.CopyFrom(reaction_pb2.Temperature(units='CELSIUS', value=60))

    # Glove box work
    p_conds = reaction.conditions.pressure
    p_conds.type = p_conds.PressureControl.SEALED
    p_conds.atmosphere = p_conds.Atmosphere.NITROGEN 
    p_conds.atmosphere_details = 'dry nitrogen'
    p_conds.details = 'prepared in a glove box'

    # Notes
    reaction.notes.safety_notes = ''
    t.stop()
    
    ###
    t = timer('Workup and outcome')
    # Standard stock solution 
    standard = stock_solution(reaction, r'External standard in DMSO')
    standard.add_solute('WORKUP', r'4,4\'-di-tert-butyl-1,1\'-biphenyl', SMILES=r'CC(C)(C)C1=CC=C(C2=CC=C(C(C)(C)C)C=C2)C=C1')
    standard.add_solvent(r'DMSO', SMILES=r'O=S(C)C', volume_liters=3e-6)
    standard.mix(concentration_molar=0.0025)

    outcome = reaction.outcomes.add()
    outcome.reaction_time.CopyFrom(unit_resolver.resolve('16 hrs'))

    # Analyses: UPLC
    # Note using LCMS because UPLC is not an option
    outcome.analyses['UPLC analysis'].type = reaction_pb2.ReactionAnalysis.LCMS
    outcome.analyses['UPLC analysis'].details = ('UPLC using 3 µL of 0.0025 M di-tert-butylbiphenyl solution in DMSO external standard')
    outcome.analyses['UPLC analysis'].instrument_manufacturer = 'Waters Acquity'

    # Define product identity
    prod_2a = outcome.products.add() 
    prod_2a.compound.identifiers.add().CopyFrom(
        reaction_pb2.CompoundIdentifier(value=prod, type='SMILES')
    )
    prod_2a.is_desired_product = True

    # Define product yield from results table
    # Yields are reported with insignificant digits
    prod_2a.compound_yield.CopyFrom(
        reaction_pb2.Percentage(value=round(y))    # Round yield to the nearest % 
    )

    # The UPLC analysis was used to confirm both identity and yield
    prod_2a.analysis_identity.append('UPLC analysis')
    prod_2a.analysis_yield.append('UPLC analysis')
    t.stop()
    
    ###
    t = timer('Provenance')
    # Reaction provenance
    reaction.provenance.city = r'Kenilworth, NJ'
    reaction.provenance.doi = r'10.1126/science.aar5169'
    reaction.provenance.publication_url = r'https://science.sciencemag.org/content/360/6385/186'
    reaction.provenance.record_created.time.value = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    reaction.provenance.record_created.person.CopyFrom(reaction_pb2.Person(
        name='Benjamin J. Shields', organization='Princeton University')
    )
    t.stop()
    
    ###
    t = timer('Validation')
    # Validate
    reaction = validations.validate_message(reaction)
    t.stop()
    
    # Append
    reactions.append(reaction)

Instantiate reaction: 0.0 s
Stock solutions: 0.000997304916381836 s
Reaction setup: 0.0 s
Workup and outcome: 0.0 s
Provenance: 0.0 s
Validation: 1.9515893459320068 s
Instantiate reaction: 0.0 s
Stock solutions: 0.0009639263153076172 s
Reaction setup: 0.0 s
Workup and outcome: 0.0 s
Provenance: 0.0 s
Validation: 1.5899813175201416 s


Validation takes by far the most time.